In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split,  cross_validate, KFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from scipy.stats import spearmanr, kruskal

In [ ]:
df = pd.read_csv('../data/clean_dataset.csv', index_col=0)

df

## Feature presentation

1) Airline: The name of the airline company is stored in the airline column. It is a categorical feature having 6 different airlines. 

2) Flight: Flight stores information regarding the plane's flight code. It is a categorical feature. 

3) Source City: City from which the flight takes off. It is a categorical feature having 6 unique cities. 

4) Departure Time: This is a derived categorical feature obtained created by grouping time periods into bins. It stores information about the departure time and have 6 unique time labels.

5) Stops: A categorical feature with 3 distinct values that stores the number of stops between the source and destination cities. 

6) Arrival Time: This is a derived categorical feature created by grouping time intervals into bins. It has six distinct time labels and keeps information about the arrival time. 

7) Destination City: City where the flight will land. It is a categorical feature having 6 unique cities. 

8) Class: A categorical feature that contains information on seat class; it has two distinct values: Business and Economy. 

9) Duration: A continuous feature that displays the overall amount of time it takes to travel between cities in hours. 

10) Days Left: This is a derived characteristic that is calculated by subtracting the trip date by the booking date. 

11) Price: Target variable stores information of the ticket price.

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#sns.pairplot(df)

In [ ]:
continuous_col = ['duration', 'days_left', 'price']
discrete_col   = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

def univariate_discrete_analysis(df, discrete_col):
    for col in discrete_col:
        print(f'### analysing column : {col} ###')
        count = df[col].value_counts()
        display(count)

        plt.figure(figsize=(8,4))
        sns.countplot(data=df, x=col, order=count.index)

        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def univariate_continuous_analysis(df, continuous_col):
    for col in continuous_col:
        print(f'### analysing column : {col} ###')
        stats = df[col].describe()
        variance = df[col].var()

        display(stats)
        print(f'Variance: {variance:.2f}')

        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        #histogram
        sns.histplot(df[col], kde=True, ax=axes[0])
        axes[0].set_title(f'Histogram of {col}')
        
        #boxplot
        sns.boxplot(x=df[col], ax=axes[1])
        axes[1].set_title(f'Boxplot of {col}')

        plt.tight_layout()
        plt.show()


    
univariate_discrete_analysis(df=df, discrete_col=discrete_col)


In [ ]:
univariate_continuous_analysis(df=df, continuous_col=continuous_col)

### Multivariate analysis

Need to do 3 levels of multivariate analysis :
- continuous  ↔ continuous
- discrete    ↔ continuous
- discrete    ↔ discrete


for continuous columns, need to check both Pearson (linear relations) & Spearman (monotone relations) 

In [ ]:
corr_pearson  = df[continuous_col].corr(method='pearson')
corr_spearman = df[continuous_col].corr(method='spearman')

print('Pearson correlations')
display(corr_pearson)
sns.heatmap(corr_pearson, annot=True, cmap='coolwarm')
plt.title('Pearson Correlation (Numerical)')
plt.show()

print('Spearman correlations')
display(corr_spearman)

sns.heatmap(corr_spearman, annot=True, cmap='coolwarm')
plt.title('Spearman Correlation (Numerical)')
plt.show()


Let's check the relation between each variable and the final flight price

##### 1. Duration VS. Price

Question : Are longer flight (in duration) associated to more expensive price  ?

Testing linear correlation between :

duration (continuous variable)
price (continuous variable)

In [ ]:
#relation between flight duration & price
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="duration", y="price", alpha=0.3)
sns.regplot(
    data=df,
    x="duration",
    y="price",
    scatter=False,
    color="red"
)
plt.title("Relation between flight duration and price")
plt.xlabel("Flight duration (hours)")
plt.ylabel("Price (Rupees)")
plt.show()


In [ ]:
df["duration_bin"] = pd.cut(
    df["duration"],
    bins=[0, 5, 10, 20, 40, 60]
)

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df,
    x="duration_bin",
    y="price"
)
plt.title("Price distribution by flight duration bins")
plt.xlabel("Flight duration (hours)")
plt.ylabel("Price (Rupees)")
plt.xticks(rotation=45)
plt.show()


<b>Hypothesis</b> : 

HO : the linear correlation between duration & price is null.

H1 : the linear correlation between duration & price is significative and not null.

In [ ]:
#Statistical correlation test between duration & price
corr_duration_price, p_value_duration_price = spearmanr(df["duration"], df["price"])

alpha = 0.02

print("Correaltion (duration, price) :", corr_duration_price)
print("p-value :", p_value_duration_price)

if p_value_duration_price < alpha:
    print(f"p-value < {alpha} : we can reject H0.")
    print("Conclusion : There is a significative linear correlation between the flight duration and the flight price.")
else:
    print(f"p-value ≥ {alpha} : we cannot reject H0.")
    print("Conclusion : There is no clear significative linear correlation between the flight duration and the flight price.")

#### 2. Air supplier VS. Price

In [ ]:
#relation between air supplier and price
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='airline', y='price')
plt.title('Relation between airline & price')
plt.xlabel('Airline')
plt.ylabel('price (in Roupies)')
plt.show()

#### 3. Amount of stops VS. Price

Does the flight price fluctuate depending the amount of stops ?

In [ ]:
#let's check basic distribution of stops
df.groupby("stops")["price"].agg(["median", "mean", "count", "std"])


In [ ]:
#relation between amount of stops & price
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='stops', y='price')
plt.title('Relation between Amount of stops & the Flight price')
plt.xlabel('Amount of stops')
plt.ylabel('Price in Roupies')
plt.show()

<b>Hypothesis</b> : 

HO - no correlation between stops & price - the average price is the same regardless of the amount of stops

H1 - the average price does vary depending on the amount of stops

In [ ]:
stops_unique = sorted(df['stops'].unique())
print('amount of stops : ', stops_unique)

groups_price_by_stops = [df[df['stops'] ==stop]['price'].values for stop in stops_unique]

print(groups_price_by_stops)

#stat test
#ANOVA not relevant because unenven group sizes & uneven variance. 
#Price is very right-skewed 

#Kruskal–Wallis does test between groups without need to normalise
h_stat_stop, p_value_stop = kruskal(*groups_price_by_stops)

print('H stat (stop VS. price) :', h_stat_stop)
print('p-value stop) :', p_value_stop)

if p_value_stop < alpha:
    print(f"p-value < {alpha} : we can reject H0.")
    print("Conclusion : There is a significative correlation between the amount of stops and the flight price. Flight price does vary depending on the amount of stops")
else:
    print(f"p-value ≥ {alpha} : we cannot reject H0.")
    print("Conclusion : There is no clear significative correlation between the amount of stops and the flight price.")


In [ ]:
#stop is an ordinal discrete variable
# #let's add to the previous statistical test another check : is there a monotonous correlation between stops & price ?
df['stop_num'] = df['stops'].map({'zero': 0, 'one': 1, 'two_or_more':2}) 

spearman_corr, p_value_spearman_stop = spearmanr(df['stop_num'], df['price'])

print(f"Spearman corr: {spearman_corr:.3f}")
print(f"p_value_spearman_stop: {p_value_spearman_stop:.5e}")

Medians do decrease between 0 and 2 stops - there is a significative negative monotonous relation validated by Spearman test.
Stops variable does have a significative impact on the price.

#### 4. Amount of Booking Lead time in days before the flight

Does booking in advance lead to cheaper flight price ?

In [ ]:
df['days_left'].unique()

df['days_bin'] = pd.cut(df['days_left'], bins = [0, 5, 10, 20, 40, 60])

plt.figure(figsize=(12,8))
sns.boxplot(data=df, x='days_bin', y='price')
plt.title('Flight price distribution in Days bins')
plt.xlabel('Bins of days in advance')
plt.ylabel('Price')

plt.show()

In [ ]:
df.groupby("days_bin")["price"].agg(["median", "mean", "count", "std"])

The average and median price do seem to significatively decrease the higher the amount of days in advance

In [ ]:
df['days_left'].unique()

df['days_bin'] = pd.cut(df['days_left'], bins = [0, 5, 10, 20, 40, 60])

plt.figure(figsize=(12,8))
sns.boxplot(data=df, x='days_bin', y='price')
plt.title('Flight price distribution in Days bins')
plt.xlabel('Bins of days in advance')
plt.ylabel('Price')

plt.show()

<b>Hypothesis</b> : 

HO : There is no significative correlation between the amount of booking lead time in days and the flight price

H1 : There is a significative linear correlation between the amount of booking lead time in days and the flight price

In [ ]:
corr_days_price, p_value_days_price = spearmanr(df['days_left'], df['price'])

print(f"Spearman correlation (days_left, price):{corr_days_price:.3f}")
print(f"p-value booking days in advance: {p_value_days_price:.5e}")

if p_value_days_price < alpha:
    print(f"p-value < {alpha} : we can reject H0.")
    print("Conclusion : There is a significative correlation between the amount of booking lead time in days and the flight price. Flight price does vary depending on the amount of booked days in advance")
else:
    print(f"p-value ≥ {alpha} : we cannot reject H0.")
    print("Conclusion : There is no clear significative correlation between the amount of booking lead time in days and the flight price.")


### Preprocessing

In [ ]:
print('NaN per columns')
df.isna().sum()

In [ ]:
#mapping classes
print(df['class'].unique())

df['class_num'] = df['class'].map({'Economy': 0, 'Business': 1})

print(df['class_num'].unique())

In [ ]:
print(df['flight'].unique())

print('-' * 30)
print('high cardinality and not informative')
print('-' * 30)

print(df["flight"].value_counts().describe())

In [ ]:
le = LabelEncoder()
flight_encoded = le.fit_transform(df['flight'])
mi_flight = mutual_info_regression(flight_encoded.reshape(-1, 1),
                                    df['price'],
                                    random_state=42
                                    )

print(('MI(flight-> price): ', mi_flight[0]))

"Flight" column has a very high Mutual Info Regression ; it is due to its very high cardinality, which pollutes results

"Flight" column is a simple flight identifier with no additional info 

In [ ]:
df['flight'].value_counts().head(50)

In [ ]:
#comparing MI to other features
features = ["airline", "class", "duration" , "stops", "days_left", "flight"]

X = df[features].apply(lambda col: col.astype("category").cat.codes)
y = df["price"]

mi = mutual_info_regression(X, y, random_state=42)

pd.Series(mi, index=features).sort_values(ascending=False)


In [ ]:
df

#### Preparing ML Train & Test split

In [ ]:
#keeping df back up
df_raw = df.copy()

# Flight column is useless
df.drop(columns='flight', inplace=True)

#dropping legacy categorical & bins non encoded columns
#del bin columns used for graph
df.drop(columns=['duration_bin', 'days_bin'], inplace=True)

#del manual encoding to do proper SKLearn encoding
df.drop(columns=['stops', 'class'], inplace=True)

In [ ]:
X = df.drop(columns='price')
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(type(X_train), X_train.shape)
print(type(X_test), X_test.shape)
print(type(y_train), y_train.shape)
print(type(y_test), y_test.shape)

In [ ]:
# splitting numeric & cat features
numeric_features     = ["duration", "days_left", "stop_num", "class_num"]
categorical_features = ["airline", "source_city", "departure_time", "arrival_time", "destination_city"]

In [ ]:
numeric_transformer = Pipeline(
                            steps=[
                                ('imputer', SimpleImputer(strategy='median')), #skewed data, using median
                                ('scaler', StandardScaler())
                                      ]
                                      )

In [ ]:
categorical_transformer = Pipeline(
                                steps=[
                                ('imputer', SimpleImputer(strategy='most_frequent')), 
                                ('OneHot', OneHotEncoder(handle_unknown='ignore', drop='first'))
                                         ])

In [ ]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                ('cat', categorical_transformer , categorical_features)
                                                ],
                                                remainder='drop')

preprocessor

In [ ]:
# fit / transform
preprocessor.fit(X_train)

X_train_proc = preprocessor.transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

X_train_proc.shape, X_test_proc.shape

### Modelisation

In [ ]:
# creating evaluating function to eval model results

def evaluate_model(
        name, 
        model,
        X_train,
        y_train,
        X_test,
        y_test,
        results_dict,
        cross_validator=5):
    """
    Evaluate a regression model using a strict experimental protocol.
    Then refit on the entire train & evaluate it once on test.

    Steps:
    1. Cross-validation on training set (performance + stability)
    2. Refit model on full training data
    3. Final evaluation on the test set
    4. Store results for later comparison

    Args:
        name (str)                      : model name
        model (estimator | Pipeline)    : Scikit-learn Model or Pipeline
        X_train , y_train (array)       : training data
        X_test, y_test (array)          : test data
        result_dict (dict)              : stocking results dict
        cross_validator (int, optional) : Amount of folds for cross validation. Defaults to 5.
    """
    
    #step 1 - cross validation on train only
    cv_results = cross_validate(
        estimator=model,
        X=X_train,
        y=y_train,
        cv=KFold(n_splits=cross_validator, shuffle=True, random_state=42),
        scoring={
            'mae'  : 'neg_mean_absolute_error',
            'rmse' : 'neg_root_mean_squared_error',
            'r2'   : 'r2'
        }
    )

    #computing absolute average - convert negative metrics back to positive values
    cv_mae_mean = abs(cv_results['test_mae'].mean())
    cv_mae_std  = abs(cv_results['test_mae'].std())

    cv_rmse_mean = abs(cv_results['test_rmse'].mean())
    cv_rmse_std  = abs(cv_results['test_rmse'].std())
    
    cv_r2_mean = cv_results['test_r2'].mean()
    cv_r2_std  = cv_results['test_r2'].std()

    print(f'-' * 30)
    print(f'{name}')
    print(f'-' * 30)

    print(f'cross validation ({cross_validator}-fold) on train:')
    print(f"  MAE  (mean ± std) : {cv_mae_mean:.2f} ± {cv_mae_std:.2f}")
    print(f"  RMSE (mean ± std) : {cv_rmse_mean:.2f} ± {cv_rmse_std:.2f}")
    print(f"  R²   (mean ± std) : {cv_r2_mean:.4f} ± {cv_r2_std:.4f}")

    #step 2 - refit back on full train set
    model.fit(X_train, y_train)

    #step 3 - final evaluation on test set
    y_test_pred = model.predict(X_test)

    test_mae  = mean_absolute_error(y_test, y_test_pred)
    test_rmse = root_mean_squared_error(y_test, y_test_pred)
    test_r2   = r2_score(y_test, y_test_pred)

    print("Final evaluation on test :")
    print(f"  Test MAE  : {test_mae:.2f}")
    print(f"  Test RMSE : {test_rmse:.2f}")
    print(f"  Test R²   : {test_r2:.4f}")
    
    results_dict[name] = {
        'cv_mae_mean'  : cv_mae_mean,
        'cv_mae_std'   : cv_mae_std,
        'cv_rmse_mean' : cv_rmse_mean,
        'cv_rmse_std'  : cv_rmse_std,
        "cv_r2_mean"   : cv_r2_mean,
        "cv_r2_std"    : cv_r2_std,
        "test_mae"     : test_mae,
        "test_rmse"    : test_rmse,
        "test_r2"      : test_r2
    }

In [ ]:
#initiating dict result
results = {}

# 1. Testing on Baseline DummyRegressor

before training on real ML models, let's test on naive baseline.

This model :
- does not test any features
- is a baseline. Any ML model must outperfom this bais DummyRegressor

else it is a uselse model

In [ ]:
dummy_model = DummyRegressor(strategy='mean')

evaluate_model(name='DummyRegressor (mean)',
               model=dummy_model,
               X_train=X_train,
               y_train=y_train,
               X_test=X_test,
               y_test=y_test,
               results_dict=results)

In [ ]:
results

# 2. LinearRgression Model test

In [ ]:
linear_reg_model = Pipeline(steps=[
                                 ('preprocessor', preprocessor),
                                 ('Linear Regression', LinearRegression())
                                 ])

evaluate_model(name='LinearRegression',
               model=linear_reg_model,
               X_train=X_train,
               y_train=y_train,
               X_test=X_test,
               y_test=y_test,
               results_dict=results,
            )

3. LinearRegression with Ridge

In [ ]:
alphas = [0.01, 0.1, 1, 10, 100]

for a in alphas:
    ridge_pipeline = Pipeline(steps=[
                                    ('preprocessor', preprocessor),
                                    ('RidgeRegression', Ridge(alpha=a))
                                ])
    
    evaluate_model(name=f'Ridge Regression (alpha={a})',
                   model=ridge_pipeline,
                   X_train=X_train,
                   y_train=y_train,
                   X_test=X_test,
                   y_test=y_test,
                   results_dict=results,
                    )

4. RandomForestRegressor

In [ ]:
rf_pipeline = Pipeline([
                        ('preprocessor', preprocessor),
                        ('RandomForest', RandomForestRegressor(n_estimators=200,
                                                               max_depth=None,
                                                               min_samples_leaf=1,
                                                               random_state=42,
                                                               n_jobs=-1 #use all CPUs

                        ))
                    ])

evaluate_model(model=rf_pipeline,
               name='RandomForestRegressor',
               X_train=X_train,
               y_train=y_train,
               X_test=X_test,
               y_test=y_test,
               results_dict=results)
